In [ ]:
import numpy as np
from tqdm import tqdm
import os

from utility import (load_data, error_divided_by_season, count, divide_dayperiod, cal_mape)
from plottingFunction import boxPlot
from new_scatter import scatterPlot

In [ ]:
path = os.path.join(os.getcwd(), "repro_data")
fileList = [os.path.join(os.getcwd(), "repro_data", i) for i in sorted(os.listdir(path)) if "avg" in i]
# model decision
names = ['PONI', 'PONI+season']
# fileList = [fileList[3], fileList[0]]
fileList

In [ ]:
data_loader = load_data(*fileList, file_num = len(fileList))
all_data, target, datetime, = data_loader._return
print(all_data[0].shape)

### Seasonal evaluation (details)

In [ ]:
tar_t = 0 # 0 or 1 or 2
# data_for_season = [x[:, 0, tar_t] for x in [all_data[0], all_data[1]]] # PONI, PONI+time
# target_for_season = target[:,tar_t] # [B, H, W]
# season_object = error_divided_by_season(data_for_season, target_for_season, datetime, [0,5,10,20,30,40,60,80])
''''''
data_for_season = [x[:, 0].sum(axis=1) for x in [all_data[0], all_data[1]]] # PONI, PONI+time
target_for_season = target.sum(axis=1) # [B, H, W]
season_object = error_divided_by_season(data_for_season, target_for_season, datetime, [0,10,30,50,70,100,150,200])
''''''
# [0=D/1=T], if D:[N_models][0=S/1=W][B, H, W]; if T:[0=S/1=W][B, H, W]
data_target_season = season_object.divide_into_seasons()
# [0=S/1=W]
time_season = season_object.time_divide()

In [ ]:
# fasion scatter heat map
scatterPlot(data_target_season[0][1][0], data_target_season[1][0], 0.5, limit=(0,400)).plot()

## fill between
- 先將資料切分成夏、冬，再配合time_season製作白天、晚上的分割
- 並討論晝夜的樣本分配

In [ ]:
summer_PONI = divide_dayperiod(data_target_season[0][0][1], time_season[1]).execute()
summer_POTI = divide_dayperiod(data_target_season[0][1][1], time_season[1]).execute()
l = [0,1,3,5,8,10,15,20,25,30,35,40,45,50]
count_1 = [count(summer_PONI[0], l), count(summer_POTI[0], l)]
count_2 = [count(summer_PONI[1], l), count(summer_POTI[1], l)]

In [ ]:
count_2[1] - count_2[0]

## Cal MAPE 

In [ ]:
# summer = 0; winter = 1
season_code = 1
n_model = 0
thsh = 100 # 大雨： >40mm/h 豪雨： >100mm/3h
interest_data = data_target_season[0][n_model][season_code]
interest_targ = data_target_season[1][season_code]

cal_mape(interest_data, interest_targ, thsh, is_larger=True, is_less=False)